In [5]:
import numpy as np
from numpy.linalg import norm
from qibo import Circuit, gates
from qibo.symbols import I, X, Y, Z
from qibo.optimizers import optimize
import time

times = np.zeros(19)

""" Returns a classical shadow of size shadow_size of the state generated by circuit, where
each snapshot consists of an array of num_qubits 2 by 2 matrices"""
def calculate_classical_shadow(circuit, shadow_size, num_qubits):

    global times 
    times[0] -= time.time()
    times[1] -= time.time()
    # Maps the Pauli measurements to the indeces 0, 1, 2 = X, Y, Z
    Pauli = [gates.X, gates.Y, gates.Z]

    # Measurement basis are chosen at random
    measurement_basis = np.random.randint(0, 3, size = (shadow_size, num_qubits))
    outcomes = np.zeros((shadow_size, num_qubits))

    times[1] += time.time()
    times[2] -= time.time()
    for i in range(shadow_size):
        # In each snapshot, we perform a measurement in a randomly chosen basis
        c = circuit.copy()
        c.add(gates.M(*range(num_qubits), basis = [Pauli[measurement_basis[i][j]] for j in range(num_qubits)]))
        outcomes[i] = c(nshots = 1).samples(binary = True)[0]
    
    times[2] += time.time()
    times[3] -= time.time()
    # Possible states in matrix form
    states = [np.array([[1, 0],[0, 0]]), np.array([[0, 0], [0, 1]])]
    # Matrices to invert the implicit unitary operations performed when measuring in different basis
    unitaries = [gates.H(0).matrix(), gates.H(0).matrix()@gates.S(0).dagger().matrix(), gates.I(0).matrix()]
    shadow = np.empty((shadow_size, num_qubits), dtype = np.ndarray)

    times[3] += time.time()
    times[4] -= time.time()
    # Computes the classical shadows for each snapshots
    for i in range(shadow_size):
        for j in range(num_qubits):
            state = states[int(outcomes[i][j])]
            U = unitaries[measurement_basis[i][j]]
            shadow[i][j] = 3*(U.conj().T @ state @ U) - np.eye(2)

    times[4] += time.time()
    times[0] += time.time()
    return shadow

"""Returns a bound on the size of a classical shadow such that the estimation of 
the observables yields a maximum error with a certain failure rate"""
def shadow_bound(observables, error = 0.01, failure_rate = 0.01, locality = 1):
    M = np.size(observables, axis = 0)
    infty_norm = max([norm(obs, ord = np.inf) for obs in observables])
    return int((4**(locality + 1))/(error**2)*np.log(2*M/failure_rate)*(infty_norm**2))

"""Creates a 2-local block for an alternating layered ansatz"""
def S_block(theta):
    c = Circuit(2)
    c.add(gates.RX(0, theta[0]))
    c.add(gates.RY(0, theta[1]))
    c.add(gates.RX(0, theta[2]))
    c.add(gates.RX(1, theta[3]))
    c.add(gates.RY(1, theta[4]))
    c.add(gates.RX(1, theta[5]))
    c.add(gates.CNOT(0, 1))
    c.add(gates.RX(0, theta[6]))
    c.add(gates.RY(0, theta[7]))
    c.add(gates.RX(0, theta[8]))
    c.add(gates.RX(1, theta[9]))
    c.add(gates.RY(1, theta[10]))
    c.add(gates.RX(1, theta[11]))
    c.add(gates.CNOT(1, 0))
    return c

"""Given a set of (num_qubits//2)*depth*12 angles, generates an alternating layered ansatz"""
def alternating_layered_ansatz(depth, num_qubits, theta):
    c = Circuit(num_qubits, density_matrix=True)
    for j in range(depth):
        for i in range(num_qubits//2):
            c.add(S_block(theta[i][j]).on_qubits((2*(i - 1) + j)%num_qubits, (2*(i - 1)+ j + 1)%num_qubits))
    return c

"""An infidelity-like 1-local cost function for the state preparation problem"""
def cost(theta, observables, reduced_shadows):

    global num_qubits, depth, times
    times[9] -= time.time()
    times[5] -= time.time()
    s = 0
    c = alternating_layered_ansatz(depth, num_qubits, theta.reshape(num_qubits//2, depth, 12))
    times[5] += time.time()
    #For maximum efficency, we exploit the locality of the observables and make use only of the necessary dimensions
    for i in range(num_qubits):
        times[6] -= time.time()
        times[17] -= time.time()
        u = c.light_cone(i)[0]
        times[17] += time.time()
        times[18] -= time.time()
        w = u + observables[i] + u.invert()
        times[18] += time.time()
        times[6] += time.time()
        times[7] -= time.time()
        s += w.expectation
        times[7] += time.time()
    times[8] -= time.time()
    print(abs(1 - s))
    times[8] += time.time()
    times[9] += time.time()
    return abs(1 - s)

times[10] -= time.time()
#Circuit parameters
num_qubits = 6
depth = 4

#Generation of random alternating layered ansatz and reduced observables
times[11] -= time.time()
rng = np.random.default_rng()
theta = rng.uniform(0, 2*np.pi, ((num_qubits//2), depth, 12))
times[12] -= time.time()
c = alternating_layered_ansatz(depth, num_qubits, theta)
times[12] += time.time()
times[13] -= time.time()
A = [c.light_cone(i)[1] for i in range(num_qubits)]
times[13] += time.time()
times[14] -= time.time()
observables = [(Z(A[i][i]).full_matrix(len(A[i])))/(2*num_qubits) for i in range(num_qubits)]
times[14] += time.time()
times[11] += time.time()

#Generation of the classical shadow and reduced shadows
shadow = calculate_classical_shadow(c, shadow_bound(observables), num_qubits)
times[15] -= time.time()
reduced_shadows = np.zeros(num_qubits, dtype = np.ndarray)
T = np.size(shadow, axis = 0)
for i in range(num_qubits):
    for j in range(T):
        hat_rho = [1/T]
        for k in list(A[i]):
            hat_rho = np.kron(hat_rho, shadow[j][k])
        reduced_shadows[i] = reduced_shadows[i] + hat_rho
times[15] += time.time()
#Optimization procedure
times[16] -= time.time()
initial_theta = np.zeros((num_qubits//2)*depth*12)
best, params, extra = optimize(cost, initial_theta, args = (observables, reduced_shadows), method = 'L-BFGS-B', options = {'maxiter': 100})
times[16] += time.time()

times[10] += time.time()

print("Ansatz and observables: ", times[11]/times[10])
print("Of which ansatz: ", times[12]/times[11])
print("A: ", times[13]/times[11])
print("Observables: ", times[14]/times[11])
print()
print("Classical shadows: ", times[0]/times[10])
print("Of which measurement setup: ", times[1]/times[0])
print("measurement: ", times[2]/times[0])
print("processing setup: ", times[3]/times[0])
print("processing: ", times[4]/times[0])
print()
print("Reduced shadows: ", times[15]/times[10])
print()
print("Optimization: ", times[16]/times[10])
print("Of which cost: ", times[9]/times[16])
print("Of which ansatz: ", times[5]/times[9])
print("light cone", times[6]/times[9])
print("(light cone):", times[17]/times[6], "(unitary):", times[18]/times[6])
print("product: ", times[7]/times[9])
print("print: ", times[8]/times[9])
print(shadow_bound(observables))
print(c.draw())

q0:     ───────────────────────RX─RY─RX─o─RX─RY─RX─X───────────────────────RX─ ...
q1:     ───────────────────────RX─RY─RX─X─RX─RY─RX─o────────────────────────── ...
q2:     ─────────────────────────────────────────────RX─RY─RX─o─RX─RY─RX─X──── ...
q3:     ─────────────────────────────────────────────RX─RY─RX─X─RX─RY─RX─o──── ...
q4:     ─RX─RY─RX─o─RX─RY─RX─X──────────────────────────────────────────────── ...
q5:     ─RX─RY─RX─X─RX─RY─RX─o─────────────────────────────────────────────RX─ ...

q0: ... RY─RX─X─RX─RY─RX─o─────────────────────────────────────────────
q1: ... ──────|──────────|─RX─RY─RX─o─RX─RY─RX─X───────────────────────
q2: ... ──────|──────────|─RX─RY─RX─X─RX─RY─RX─o───────────────────────
q3: ... ──────|──────────|───────────────────────RX─RY─RX─o─RX─RY─RX─X─
q4: ... ──────|──────────|───────────────────────RX─RY─RX─X─RX─RY─RX─o─
q5: ... RY─RX─o─RX─RY─RX─X─────────────────────────────────────────────
0 : 
q0: ───────────────────────RX─RY─RX─o─RX─RY─RX─X─RX─RY─RX─X─RX─R